In [3]:
from Scripts.import_data import *
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor
import numpy as np

Zaimportowanie wartości rynkowej

In [4]:
#market_values to procentowe roczne zmiany wartości spółek, obliczone w notebooku "Market_value_importer"
market_values = pd.read_csv('./Data/market_values.csv')
market_values.set_index('Unnamed: 0', inplace=True)

Zaimportowanie danych dotyczących alokacji środków

In [5]:
balance_data = import_csv_dict_of_dataframes('./Data/companies_balance_sheet.csv')

for company, years in balance_data.items():
    for year, data in years.items():
        if company in market_values.index and year in market_values.columns:
            # Take only the second level of the MultiIndex
            data.index = data.index.droplevel(0)

Stworzenie jednego zbioru z wszystkimi danymi

In [6]:
columns = ['ASSETS','Property, plant and equipment','Exploration for and evaluation of mineral resources','Intangible assets','Goodwill','Investment property',
'Right-of-use assets','Investment in affiliates','Non-current financial assets','Non-current loans and receivables','Deferred income tax','Non-current deferred charges and accruals',
'Non-current derivative instruments','Other non-current assets','Inventories','Current intangible assets',
'Biological assets','Trade receivables','Loans and other receivables','Financial assets','Cash and cash equivalents','Accruals','Assets from current tax','Derivative instruments',
'Other assets','Assets held for sale and discontinuing operations','Called up capital','Own shares','EQUITY & LIABILITIES','Equity shareholders of the parent','Share capital',
'Called up share capital','Treasury shares','Supplementary capital','Valuation and exchange differences','Other capitals','Retained earnings / accumulated losses','Non-controlling interests',
'Non-current liabilities','Non-current liabilities from derivatives','Non-current loans and borrowings','Non-current liabilities from bonds','Non-current liabilities from finance leases',
'Non-current trade payables','Long-term provision for employee benefits','Deferred tax liabilities','Non-current provision','Other non-current liabilities','Non-current accruals (liability)',
'Current liabilities','Liabilities from derivatives','Financial liabilities (loans and borrowings)','Bond liabilities','Liabilities from finance leases','Trade payables','Employee benefits',
'Current tax liabilities','Provisions','Other liabilities','Accruals (liability)','Liabilities related to assets held for sale and discontinued operations', 'market_value']

rows = []
for company, years in balance_data.items():
    for year, market_data in years.items():
        new_row = {}
        for indicator, value in market_data.items():
            if indicator in columns:
                new_row[indicator] = value
            else:
                new_row[indicator] = 0
        if company in market_values.index and year in market_values.columns:
            new_row['market_value'] = market_values[year][company]
            rows.append(new_row)

data = pd.DataFrame(rows, columns=columns)
data

,ASSETS,"Property, plant and equipment",Exploration for and evaluation of mineral resources,Intangible assets,Goodwill,Investment property,Right-of-use assets,Investment in affiliates,Non-current financial assets,Non-current loans and receivables,...,Bond liabilities,Liabilities from finance leases,Trade payables,Employee benefits,Current tax liabilities,Provisions,Other liabilities,Accruals (liability),Liabilities related to assets held for sale and discontinued operations,market_value
0,48649.817,814.332,NaN,13067.498,NaN,NaN,NaN,NaN,3053.534,NaN,...,NaN,NaN,1197.853,NaN,NaN,NaN,NaN,466.396,NaN,107.934581
1,109262.821,18734.064000000002,NaN,17138.011,NaN,NaN,NaN,NaN,5094.282,NaN,...,NaN,NaN,5122.519,NaN,NaN,NaN,NaN,758.971,NaN,39.139118
2,155668.388,20380.205,NaN,24870.505,NaN,NaN,4007.699,NaN,NaN,NaN,...,NaN,12.653,5494.21,NaN,556.548,NaN,41.795,17256.979,NaN,23.231833
3,186339.222,26889.503,NaN,37859.517,NaN,NaN,4003.398,NaN,NaN,NaN,...,NaN,12.651,7444.283,NaN,1184.903,860.272,41.796,320.698,NaN,63.933486
4,219512.964,25946.479,NaN,59137.637,NaN,NaN,4281.361,NaN,NaN,NaN,...,NaN,14.734,6332.759,NaN,744.927,661.422,41.796,4169.079,NaN,22.300192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1240,1778284,809413,NaN,36338,NaN,NaN,NaN,58102,NaN,20899,...,NaN,NaN,913510,4258,39660,10012,NaN,NaN,NaN,14.618363
1241,1790063,818103,NaN,29100,NaN,NaN,NaN,58102,NaN,22286,...,NaN,NaN,1011972,2327,2528,11769,NaN,NaN,NaN,6.546306
1242,2479535,905788,NaN,26674,NaN,NaN,35897,532325,NaN,11005,...,NaN,6488,704176,628,76761,8900,NaN,321194,NaN,-2.118575
1243,2513606,1053811,NaN,360038,NaN,NaN,83331,58102,NaN,52234,...,NaN,7175,586113,204,103458,11302,NaN,350508,NaN,6.060657


Czyszczenie danych

In [7]:
data = data.fillna(0)
data = data.apply(pd.to_numeric, errors='coerce')
data = data[data['ASSETS'] > 0]
data = data[data['market_value'] > 0]
y = data['market_value']
X = data.drop('market_value', axis=1)
X

,ASSETS,"Property, plant and equipment",Exploration for and evaluation of mineral resources,Intangible assets,Goodwill,Investment property,Right-of-use assets,Investment in affiliates,Non-current financial assets,Non-current loans and receivables,...,Financial liabilities (loans and borrowings),Bond liabilities,Liabilities from finance leases,Trade payables,Employee benefits,Current tax liabilities,Provisions,Other liabilities,Accruals (liability),Liabilities related to assets held for sale and discontinued operations
0,48649.817,814.332,0,13067.498,0.0,0.0,0.000,0.0,3053.534,0.0,...,0.000,0.0,0.000,1197.853,0.0,0.000,0.000,0.000,466.396,0.0
1,109262.821,18734.064,0,17138.011,0.0,0.0,0.000,0.0,5094.282,0.0,...,1452.203,0.0,0.000,5122.519,0.0,0.000,0.000,0.000,758.971,0.0
2,155668.388,20380.205,0,24870.505,0.0,0.0,4007.699,0.0,0.000,0.0,...,1303.717,0.0,12.653,5494.210,0.0,556.548,0.000,41.795,17256.979,0.0
3,186339.222,26889.503,0,37859.517,0.0,0.0,4003.398,0.0,0.000,0.0,...,1260.000,0.0,12.651,7444.283,0.0,1184.903,860.272,41.796,320.698,0.0
4,219512.964,25946.479,0,59137.637,0.0,0.0,4281.361,0.0,0.000,0.0,...,1260.000,0.0,14.734,6332.759,0.0,744.927,661.422,41.796,4169.079,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1234,758826.000,357311.000,0,42626.000,0.0,2442.0,0.000,0.0,2172.000,7270.0,...,68920.000,0.0,16559.000,192112.000,0.0,0.000,14003.000,2761.000,995.000,0.0
1235,475282.000,92004.000,0,8594.000,31172.0,9095.0,0.000,210.0,0.000,0.0,...,8809.000,0.0,0.000,205950.000,16184.0,0.000,2998.000,6663.000,475.000,0.0
1240,1778284.000,809413.000,0,36338.000,0.0,0.0,0.000,58102.0,0.000,20899.0,...,97060.000,0.0,0.000,913510.000,4258.0,39660.000,10012.000,0.000,0.000,0.0
1241,1790063.000,818103.000,0,29100.000,0.0,0.0,0.000,58102.0,0.000,22286.0,...,418770.000,0.0,0.000,1011972.000,2327.0,2528.000,11769.000,0.000,0.000,0.0


In [8]:
# zamiana na procenty
X_normalized = X.copy()

# ASSETS
columns_to_divide = X.columns[1:63]
X_normalized.iloc[:, 1:63] = X_normalized.iloc[:, 1:63].div(X['ASSETS'], axis=0)

In [9]:
# Detekcja i usuwanie anomalii

def delete_anomaly(X: pd.DataFrame, y: pd.DataFrame, threshold: float) -> pd.DataFrame:
    # Calculate the mean for each column except the first one
    mean_val = X[X.columns[1:63]].mean()
    
    # Calculate the absolute deviation from the mean for each column except the first one
    deviation = X[X.columns[1:63]].sub(mean_val).abs()

    # Create a boolean DataFrame where each element is True if it meets the condition
    condition = (deviation <= threshold)
    # Create a mask for rows where all columns meet the condition
    row_mask = condition.all(axis=1)
    # Filter the DataFrame using the row mask
    X_filtered = X[row_mask]
    y_filtered = y[row_mask]
    
    return X_filtered, y_filtered

In [10]:
X_norm, y_norm = delete_anomaly(X_normalized, y, 1000000000)
X_norm = X_norm.drop('ASSETS', axis=1)

Uczenie modelu

In [12]:
X_norm

,"Property, plant and equipment",Exploration for and evaluation of mineral resources,Intangible assets,Goodwill,Investment property,Right-of-use assets,Investment in affiliates,Non-current financial assets,Non-current loans and receivables,Deferred income tax,...,Financial liabilities (loans and borrowings),Bond liabilities,Liabilities from finance leases,Trade payables,Employee benefits,Current tax liabilities,Provisions,Other liabilities,Accruals (liability),Liabilities related to assets held for sale and discontinued operations
0,0.016739,0.0,0.268603,0.000000,0.000000,0.000000,0.000000,0.062766,0.000000,0.001850,...,0.000000,0.0,0.000000,0.024622,0.000000,0.000000,0.000000,0.000000,0.009587,0.0
1,0.171459,0.0,0.156851,0.000000,0.000000,0.000000,0.000000,0.046624,0.000000,0.001446,...,0.013291,0.0,0.000000,0.046883,0.000000,0.000000,0.000000,0.000000,0.006946,0.0
2,0.130921,0.0,0.159766,0.000000,0.000000,0.025745,0.000000,0.000000,0.000000,0.003482,...,0.008375,0.0,0.000081,0.035294,0.000000,0.003575,0.000000,0.000268,0.110857,0.0
3,0.144304,0.0,0.203175,0.000000,0.000000,0.021484,0.000000,0.000000,0.000000,0.008144,...,0.006762,0.0,0.000068,0.039950,0.000000,0.006359,0.004617,0.000224,0.001721,0.0
4,0.118200,0.0,0.269404,0.000000,0.000000,0.019504,0.000000,0.000000,0.000000,0.004362,...,0.005740,0.0,0.000067,0.028849,0.000000,0.003394,0.003013,0.000190,0.018992,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1234,0.470873,0.0,0.056174,0.000000,0.003218,0.000000,0.000000,0.002862,0.009581,0.005004,...,0.090825,0.0,0.021822,0.253170,0.000000,0.000000,0.018454,0.003639,0.001311,0.0
1235,0.193578,0.0,0.018082,0.065586,0.019136,0.000000,0.000442,0.000000,0.000000,0.016418,...,0.018534,0.0,0.000000,0.433322,0.034051,0.000000,0.006308,0.014019,0.000999,0.0
1240,0.455165,0.0,0.020434,0.000000,0.000000,0.000000,0.032673,0.000000,0.011752,0.039225,...,0.054581,0.0,0.000000,0.513703,0.002394,0.022302,0.005630,0.000000,0.000000,0.0
1241,0.457025,0.0,0.016256,0.000000,0.000000,0.000000,0.032458,0.000000,0.012450,0.048570,...,0.233941,0.0,0.000000,0.565328,0.001300,0.001412,0.006575,0.000000,0.000000,0.0


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_norm, y_norm, test_size=0.2, random_state=42)

models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'Support Vector Machine': SVR(kernel='linear'),
    'Neural Network': MLPRegressor(hidden_layer_sizes=(100,), activation='relu', solver='adam', max_iter=1000)
}

# Train and evaluate each model
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f'{name} MSE: {mse}')

Linear Regression MSE: 549895.9920622634
Random Forest MSE: 477445.8201508186
Gradient Boosting MSE: 531960.0456770387
Support Vector Machine MSE: 605207.0522089173
Neural Network MSE: 590993.9541400878


In [31]:
import random
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")
model = models['Random Forest']

def evaluate_market_value_change(property_structure, model):
    structure = np.array(property_structure).reshape(1, -1)
    return model.predict(structure)[0]

population_size = 100
num_generations = 200
mutation_rate = 0.05
num_properties = 60

def initialize_population(size, num_properties):
    random_values = [np.random.rand(num_properties) for _ in range(size)]
    sum = np.sum(random_values)
    return [i/sum for i in random_values] #we divide random numbers by sum so that capital property in percentages sums up to 100%

def evaluate_population(population, model):
    return [evaluate_market_value_change(individual, model) for individual in population]

def select_parents(population, fitness, num_parents):
    fitness = np.array(fitness)
    fitness_probs = fitness / np.sum(fitness)
    print(fitness_probs)
    parent_indices = np.random.choice(range(len(population)), size=num_parents, p=fitness_probs)
    parents = [population[i] for i in parent_indices]
    return parents

def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
    sumchild1 = np.sum(child1)
    sumchild2 = np.sum(child2)
    child1 = [i/sumchild1 for i in child1] # scaling to make sure that capital properties sum up to 100%
    child2 = [i/sumchild2 for i in child2]
    return child1, child2

def mutate(individual, mutation_rate):
    for i in range(len(individual)):
        if random.random() < mutation_rate:
            individual[i] = np.random.rand()
    individual_sum = np.sum(individual)
    individual = [i/individual_sum for i in individual]  # scaling to make sure that capital properties sum up to 100%
    return individual

def select_next_generation(population, fitness, num_offspring):
    sorted_population = [x for _, x in sorted(zip(fitness, population), reverse=True)]
    return sorted_population[:num_offspring]

def evolutionary_algorithm(num_generations, population_size, num_properties, mutation_rate, model):
    population = initialize_population(population_size, num_properties)
    
    for generation in range(num_generations):
        fitness = evaluate_population(population, model)
        
        parents = select_parents(population, fitness, population_size // 2)
        offspring = []
        
        for i in range(0, len(parents), 2):
            parent1, parent2 = parents[i], parents[i-1]
            child1, child2 = crossover(parent1, parent2)
            offspring.append(mutate(child1, mutation_rate))
            offspring.append(mutate(child2, mutation_rate))
        
        population = select_next_generation(population + offspring, fitness + evaluate_population(offspring, model), population_size)
        
        best_fitness = max(fitness)
        best_individual = population[np.argmax(fitness)]
        print(f'Generation {generation + 1}, Best Fitness: {best_fitness}, Best Individual: {best_individual}')
    
    return population[np.argmax(fitness)]

In [32]:
best_property_structure = evolutionary_algorithm(num_generations, population_size, num_properties, mutation_rate, model)
print('Best Property Structure:', best_property_structure)

[0.0110903  0.0144125  0.0107978  0.00732218 0.01120999 0.01095128
 0.01127631 0.00752992 0.01135092 0.00761649 0.00734162 0.01095389
 0.0076222  0.01110079 0.00740778 0.01440129 0.01092293 0.0108293
 0.00771161 0.007461   0.00751657 0.00781573 0.01083808 0.00751256
 0.01093856 0.011084   0.01446472 0.00764518 0.00739063 0.00736291
 0.00739709 0.01084192 0.01092654 0.01084064 0.01110565 0.01080622
 0.00733461 0.01114045 0.01112911 0.01075033 0.00737057 0.01433997
 0.00740884 0.01097209 0.01063673 0.0077565  0.01082705 0.01082258
 0.00726423 0.00733968 0.01073593 0.01082268 0.01117531 0.00795125
 0.01084327 0.00777362 0.00735596 0.00764498 0.01148397 0.00737958
 0.01071131 0.01091139 0.01111219 0.01108588 0.01099311 0.01452639
 0.01082057 0.01425191 0.01113709 0.01107192 0.00777679 0.00783897
 0.00750476 0.00751721 0.01100534 0.00746737 0.01068204 0.01443053
 0.01089277 0.01090316 0.01129341 0.01447947 0.01443034 0.00741376
 0.01459099 0.01090985 0.00736959 0.00807494 0.0109106  0.01085